In [ ]:
import datetime



Unable to get stock data from yahoo with pandas_datareader

https://stackoverflow.com/questions/68327968/unable-to-get-stock-data-from-yahoo-with-pandas-datareader/68435757#68435757

In [ ]:
!pip install --upgrade pandas
!pip install --upgrade pandas-datareader



In [ ]:
import pandas as pd
import pandas_datareader as pdr



In [ ]:
end_date = pd.Timestamp.today()
start_date = end_date + datetime.timedelta(weeks=-1)



Python for Finance, Part I: Yahoo & Google Finance API, pandas, and matplotlib
https://www.learndatasci.com/tutorials/python-finance-part-yahoo-finance-api-pandas-matplotlib/

An Even Easier Guide to Getting Stock Data With Python
https://towardsdatascience.com/an-even-easier-guide-to-getting-stock-data-with-python-1a109df6b593



Google Finance API and Its Alternatives (Updated for 2021) https://rapidapi.com/blog/google-finance-api-alternatives/

In [ ]:
def check_version(req=[0, 10, 0], pdr_version:str=pdr.__version__):
  v0, v1, v2 = pdr_version.split('.')

  result = False

  if int(v0) > req[0]:
    result = True
  elif int(v1) > req[1]:
    result = True
  elif int(v2) >= req[2]:
    result = True

  return result



Test function `check_version()` before using



In [ ]:
assert check_version([0, 9, 9], "0.10.0")



In [ ]:
assert check_version([0, 10, 0], "0.10.0")



Check `pandas-datareader` version


In [ ]:
assert check_version([0, 10, 0], pdr.__version__), pdr.__version__



In [ ]:
# works with pandas 1.3.5 and pandas-datareader 0.10.0
panel_data = pdr.data.DataReader("SPY", "yahoo", start_date, end_date)



In [ ]:
panel_data

